In [31]:
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [32]:
def audio_norm(data):
    max_data = np.max(np.absolute(data))
    return data/(max_data+1e-6)*0.5

In [33]:
class Config(object):
    def __init__(self,
                 sampling_rate=16000, audio_duration=4, n_classes=41,
                 use_mfcc=False, n_folds=10, learning_rate=0.0001, 
                 max_epochs=50, n_mfcc=40, datagen_num = 2):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_mfcc = use_mfcc
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.datagen_num = datagen_num
        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)

In [34]:
def prepare_data(df, config, data_dir):
    X = np.empty(shape=(df.shape[0] * config.datagen_num, config.dim[0], config.dim[1], 1))
    y = np.empty(df.shape[0] * config.datagen_num)
    input_length = config.audio_length
    for i, fname in enumerate(df.index):
        print(fname+' ({0}/{1})'.format(i,df.shape[0]))
        file_path = data_dir + fname
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate)
        data = audio_norm(data)
        for j in range(config.datagen_num):
            shifted_data = data
            #pitch shift
            #bpo = 24 #how many steps per octave
            #pr = 3/24 #pitch shift range
            #ps = int(np.random.uniform(-pr * bpo, pr * bpo) + 0.5) #how many (fractional) half-steps to shift y
            #shifted_data = librosa.effects.pitch_shift(shifted_data, config.sampling_rate, n_steps = ps, bins_per_octave = bpo)
            # time stretch
            tr = 1.1 #speed up/down rate
            lgtr = np.log(tr)
            ts = 2 ** np.random.uniform(-lgtr,lgtr)
            shifted_data = librosa.effects.time_stretch(shifted_data, ts)
            #white noise
            #wnvr = 0.05 # white noise volume range
            #wnv  = np.random.uniform(0, wnvr) # white noise volume, random
            #shifted_data += np.random.uniform(-wnv, wnv, shifted_data.shape)
            # Random offset / Padding
            if len(shifted_data) < input_length:
                ratio = input_length/len(shifted_data)
                ratio = np.floor(ratio)+1
                shifted_data = np.tile(shifted_data,int(ratio))
            max_offset = len(shifted_data) - input_length
            offset = np.random.randint(max_offset)
            shifted_data = shifted_data[offset:(input_length+offset)]
            #mfcc
            shifted_data = librosa.feature.mfcc(shifted_data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
            shifted_data = np.expand_dims(shifted_data, axis=-1)
            X[ i * config.datagen_num + j, :] = shifted_data
            y[ i * config.datagen_num + j] = df.label_idx[i]
    return X, y

In [35]:
def prepare_test_data(config, data_dir='../input/audio_test/'):
    df = pd.read_csv('../input/test.csv')
    test_data = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    input_length = config.audio_length
    for i, fname in enumerate(df['fname']):
        print(fname)
        file_path = data_dir + fname
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate)
        if len(data)==0:
            data = np.zeros(88200)
        data = audio_norm(data)
        # Random offset / Padding
        if len(data) <= input_length:
            ratio = input_length/len(data)
            ratio = np.floor(ratio)+1
            data = np.tile(data,int(ratio))
        max_offset = len(data) - input_length
        offset = np.random.randint(max_offset)
        data = data[offset:(input_length+offset)]
        data = librosa.feature.mfcc(data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
        data = np.expand_dims(data, axis=-1)
        test_data[i] = data
    return test_data

In [36]:
config = Config(sampling_rate=44100, audio_duration=2, n_folds=5, 
                learning_rate=0.001, use_mfcc=True, n_mfcc=40, datagen_num=5)
train = pd.read_csv("../input/train.csv")
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index("fname", inplace=True)
train["label_idx"] = train.label.apply(lambda x: label_idx[x])

In [37]:
test_data = prepare_test_data(config, data_dir='../input/audio_test/')

00063640.wav
0013a1db.wav
002bb878.wav
002d392d.wav
00326aa9.wav
0038a046.wav
003995fa.wav
005ae625.wav
007759c4.wav
008afd93.wav
00a161c0.wav
00a7a2f6.wav
00ae03f6.wav
00b2404e.wav
00beb030.wav
00c4d5b8.wav
00c92c05.wav
00ccf065.wav
00d0ab77.wav
00dffe3a.wav
00e33205.wav
00e7ed07.wav
00eac343.wav
00f2f692.wav
0102a895.wav
010a0b3a.wav
010eed01.wav
01115622.wav
01198b7e.wav
01207ce5.wav
0137bfba.wav
014eb52e.wav
01608fa3.wav
0169348e.wav
016e52da.wav
0183b482.wav
0185de6b.wav
01a5a2a3.wav
01aed32a.wav
01afccc5.wav
01b4cdb5.wav
01b85217.wav
01bb344f.wav
01bede12.wav
01c9d6d3.wav
01e628cb.wav
01e6e112.wav
01e99e2d.wav
01eccb6e.wav
01f27f99.wav
01f29edb.wav
01f9883e.wav
01fd22ba.wav
01fe4a01.wav
02107093.wav
02182ebd.wav
02198549.wav
0220d48a.wav
02210ee7.wav
022170df.wav
023431b2.wav
023e7db3.wav
023eab1f.wav
02482fdc.wav
024e45b1.wav
026820e6.wav
02697329.wav
027a8b14.wav
027dbe3e.wav
027f06cc.wav
028278f0.wav
028db587.wav
02960f07.wav
0298b2c6.wav
02a0eb3c.wav
02ad476f.wav
02af55f6.wav

1141793d.wav
11442d0e.wav
1146e063.wav
11544687.wav
115a258b.wav
115aa41f.wav
116b77d5.wav
11704d14.wav
11770392.wav
117dcac8.wav
11844b37.wav
11867b61.wav
1189cf83.wav
118f1763.wav
11986117.wav
119a3278.wav
11a10ce8.wav
11a4f814.wav
11ae787e.wav
11cc6d11.wav
11e6a1ec.wav
11e80486.wav
11e95378.wav
11edc852.wav
11f661bd.wav
11f6b674.wav
11fa11a0.wav
12034b1a.wav
120c7292.wav
1221a798.wav
122abe8c.wav
122f1508.wav
12394644.wav
123986b6.wav
1242ac46.wav
1248a779.wav
1248f9e2.wav
12508a7b.wav
12584f90.wav
125c89cb.wav
125dccd9.wav
12679aee.wav
126a48d2.wav
1271e7aa.wav
12770ffc.wav
12798b6a.wav
127a43ec.wav
127d3803.wav
12a0327a.wav
12a83756.wav
12ac9a66.wav
12adec75.wav
12ae3ff9.wav
12bb4b63.wav
12bf9735.wav
12bfa703.wav
12c73c11.wav
12d71e7f.wav
12d94273.wav
12e91a56.wav
12ed3971.wav
12edfe79.wav
12f5ec05.wav
12f71a9d.wav
12f9efda.wav
12fb2e35.wav
130112d1.wav
1302eade.wav
130988c6.wav
13184046.wav
13331371.wav
13354f83.wav
133599c8.wav
13371a0b.wav
133a1dac.wav
13489639.wav
138bb73e.wav

225d9633.wav
225e3135.wav
225f0626.wav
2261073e.wav
226ea2c6.wav
227f2c50.wav
228153f2.wav
22824c18.wav
22891802.wav
2293e34d.wav
22abde0a.wav
22b2efa2.wav
22b5a548.wav
22c3952c.wav
22c798ac.wav
22ca2dbb.wav
22e88b33.wav
22f3b36c.wav
22f70afe.wav
23029663.wav
23071dd2.wav
23094c55.wav
230ce5d5.wav
23176e01.wav
231d230c.wav
231e9e57.wav
231f314d.wav
232750a8.wav
232ec97b.wav
23300ded.wav
233996cf.wav
233a83b7.wav
233d0591.wav
233d8cbb.wav
235dbbf0.wav
2369256d.wav
23732830.wav
23824542.wav
23852f22.wav
238d7b37.wav
238d8cad.wav
238deaad.wav
23984d44.wav
239988bc.wav
2399acef.wav
23bae3b1.wav
23bf31da.wav
23ca551e.wav
23cdfc67.wav
23d1d920.wav
23d4cbff.wav
23e94e4f.wav
23ede927.wav
23f2b1f7.wav
23f592dd.wav
23f8b4b0.wav
23fc49d9.wav
240c45cb.wav
240d2bb2.wav
240dfb34.wav
241224d6.wav
242118f2.wav
2421d14b.wav
2423598a.wav
2429265b.wav
242ea92c.wav
2436090e.wav
243ab0b7.wav
2442e7c6.wav
244618d7.wav
244baebc.wav
244dc716.wav
24599c96.wav
24602db6.wav
2472cdfe.wav
247431d8.wav
247a90a0.wav

34c7a37b.wav
34cddac7.wav
34d1fffa.wav
34d84768.wav
34f6f431.wav
34ff2f37.wav
350b89b8.wav
350ed42d.wav
35115db0.wav
351b0ba1.wav
351dcc25.wav
35235d3f.wav
352c769d.wav
352f143a.wav
3531e0f4.wav
3533dc16.wav
3538f800.wav
353b44b7.wav
353f55db.wav
3542eddc.wav
35497fc4.wav
355eeedf.wav
355f0738.wav
356cab15.wav
3579ff4b.wav
3592e0cb.wav
3597c4e5.wav
3598cc58.wav
359c9ba3.wav
35b91bb1.wav
35bb05b0.wav
35bf4ae5.wav
35c0f6b7.wav
35cb8bd1.wav
35ce58ae.wav
35d008df.wav
35d0858d.wav
35da43f9.wav
35def6dc.wav
35e1140b.wav
35e4b0fd.wav
35f3a48c.wav
35f6a4b8.wav
35ff3f29.wav
35ffae5a.wav
360348af.wav
360e9c8e.wav
360ed752.wav
361285ac.wav
3617db48.wav
361c028f.wav
361fbd17.wav
3620c12b.wav
3623b02e.wav
363a3760.wav
364d2e90.wav
36507474.wav
365a4c89.wav
365f9044.wav
3660d731.wav
3668b282.wav
36690c27.wav
3670cc04.wav
367ba81f.wav
36a0c334.wav
36a2202d.wav
36a92b29.wav
36a98063.wav
36b4867e.wav
36ba1f72.wav
36cd9e16.wav
36d47091.wav
36dbdb68.wav
36e470e4.wav
36e7790c.wav
36e79aec.wav
36eae2b1.wav

46321665.wav
46380229.wav
463d4298.wav
463de778.wav
463e231d.wav
4646a6e3.wav
464ae66b.wav
465c6192.wav
46622a4c.wav
466647df.wav
4666bee5.wav
466e8bab.wav
467c5a5a.wav
4683f13c.wav
4698193e.wav
4699c352.wav
46a20346.wav
46a24375.wav
46a4d46b.wav
46ab7601.wav
46aebc33.wav
46b06055.wav
46b5a369.wav
46d15a22.wav
46e1f70a.wav
46f07933.wav
46f0b51b.wav
4706cebd.wav
471a6fca.wav
471ab81d.wav
47268a0a.wav
47333aac.wav
4733f511.wav
473bd952.wav
473e9a47.wav
4741ae62.wav
47494a3a.wav
47538a81.wav
47542ddc.wav
47744c05.wav
47793988.wav
47795659.wav
47913bc5.wav
4797fc8b.wav
479ab3cf.wav
479d9994.wav
47a26734.wav
47ac630d.wav
47acbfcd.wav
47af6236.wav
47b7eb95.wav
47bba234.wav
47c48965.wav
47c57028.wav
47ca6f5b.wav
47e5c682.wav
47eba90e.wav
47ebd5f5.wav
47f5e3e1.wav
47f7b3fa.wav
47fa98ea.wav
480fb007.wav
481623f9.wav
48165dd3.wav
481c84da.wav
481f3844.wav
48395d1c.wav
483c028d.wav
483feb77.wav
48432e76.wav
484e69ce.wav
484e96f4.wav
4854915c.wav
486c058b.wav
487673e8.wav
487c9504.wav
487d8ae9.wav

...

ac822579.wav
ac8b9268.wav
ac8c9f4c.wav
ac8cce64.wav
ac909b62.wav
ac9151cf.wav
ac916661.wav
ac9f8827.wav
aca050ea.wav
aca46609.wav
acb2260c.wav
acb2ae9a.wav
acb33e73.wav
acb60d34.wav
acba91ee.wav
accb746f.wav
acd9adb9.wav
acdcfb8f.wav
ace88278.wav
acf4569a.wav
acf4890c.wav
ad0ab6fa.wav
ad142ed0.wav
ad1a8bce.wav
ad1f2187.wav
ad23f99c.wav
ad2ee4b2.wav
ad323692.wav
ad4c828c.wav
ad4d97de.wav
ad5821b3.wav
ad617575.wav
ad6e3b22.wav
ad71a0ba.wav
ad72561f.wav
ad7b2dfe.wav
ad7bd363.wav
ad7dbf27.wav
ad814adf.wav
ad8a6cdd.wav
ad8b78e2.wav
ad8d702a.wav
ad8de222.wav
ad936e6a.wav
ada15329.wav
ada2b04f.wav
adbf84f5.wav
adc41a55.wav
add0a091.wav
add0efb3.wav
add3bd1d.wav
add600ff.wav
ade59799.wav
adf7b2fa.wav
ae17caec.wav
ae1e828a.wav
ae23366f.wav
ae25e2b8.wav
ae2e8d4f.wav
ae302df0.wav
ae331164.wav
ae374569.wav
ae3dc480.wav
ae43398f.wav
ae48e08f.wav
ae4e8c4f.wav
ae4fbd56.wav
ae57c6d1.wav
ae58af6d.wav
ae62b053.wav
ae6c0369.wav
ae7ce8c2.wav
ae7cf02a.wav
ae85c8f5.wav
ae923dbb.wav
ae9736a3.wav
ae980378.wav

bd575d0a.wav
bd58d191.wav
bd5e02e4.wav
bd63ed5f.wav
bd661885.wav
bd6b558b.wav
bd6c1270.wav
bd702317.wav
bd70a3ae.wav
bd772d30.wav
bd789949.wav
bd79cbb8.wav
bd8a0d61.wav
bd9cf441.wav
bdb3d952.wav
bdbb5a2a.wav
bdc28adc.wav
bdc2a6f2.wav
bdc3a110.wav
bdc5b4a7.wav
bdd0b24e.wav
bdd2a664.wav
bdd2f431.wav
bdd4e518.wav
bdee019c.wav
bdf92503.wav
be09bb9d.wav
be0c6f8d.wav
be12134a.wav
be176425.wav
be264bf2.wav
be2ea892.wav
be437c69.wav
be46aa9e.wav
be4c1d3d.wav
be4f04a8.wav
be53a71f.wav
be571523.wav
be640229.wav
be666047.wav
be66ee27.wav
be7277bc.wav
be771343.wav
be83e54e.wav
be8b491b.wav
be8c3977.wav
be8f6c7c.wav
be8fcd90.wav
be92929b.wav
be9482c7.wav
be95f53a.wav
be977e89.wav
be97f271.wav
be9d0575.wav
bea3dcfa.wav
bea9dbc3.wav
bebf2559.wav
bec4f3ab.wav
bec6e952.wav
bed3738c.wav
bedf541e.wav
bee10a06.wav
bee40633.wav
bee570da.wav
bee58101.wav
bee64b40.wav
bee6c09a.wav
bee82f09.wav
bef2e832.wav
bef40206.wav
bef415f6.wav
bf05bec8.wav
bf0dc1b9.wav
bf11239b.wav
bf1255dc.wav
bf12e1fd.wav
bf14d7eb.wav

ceab0387.wav
ceb6e9c1.wav
cec9647b.wav
cecb67c0.wav
cecd4f9f.wav
cee05a5f.wav
cee063a7.wav
ceec151c.wav
ceef96eb.wav
cefb71c5.wav
ceffb395.wav
cf0471b4.wav
cf0d3082.wav
cf0f872c.wav
cf109573.wav
cf156f7d.wav
cf225634.wav
cf2c1168.wav
cf370066.wav
cf3933c7.wav
cf3f6594.wav
cf4eb36a.wav
cf510fc8.wav
cf583cf8.wav
cf5ec323.wav
cf654f37.wav
cf655c72.wav
cf6a21d7.wav
cf6c9166.wav
cf6e8727.wav
cf7b4e97.wav
cf830a16.wav
cf8357ba.wav
cf83e7ed.wav
cf99e424.wav
cf9fa471.wav
cfada6b4.wav
cfadac1f.wav
cfb02ecb.wav
cfb76128.wav
cfc16683.wav
cfc20796.wav
cfc2388d.wav
cfd05410.wav
cfd7f8d1.wav
cfe26f15.wav
cfe27a89.wav
cfe540af.wav
cfeb4cda.wav
cfee75b9.wav
cfef6dc7.wav
d001e455.wav
d002c5a8.wav
d008405a.wav
d012cd6f.wav
d012d82d.wav
d014078b.wav
d01571db.wav
d0296d9a.wav
d02c2598.wav
d0331f5c.wav
d033f19a.wav
d034f755.wav
d037cc6b.wav
d0420016.wav
d04b6cac.wav
d0564f2e.wav
d056aad7.wav
d061c5bf.wav
d06d9b40.wav
d07109a6.wav
d074e2de.wav
d076438e.wav
d07f1b62.wav
d0893bc2.wav
d09374f6.wav
d09ea1ea.wav

e0457b4a.wav
e059d9a7.wav
e06b1539.wav
e06b693c.wav
e06bfc3d.wav
e06deab0.wav
e0802994.wav
e083ae15.wav
e08808ae.wav
e08fced0.wav
e0a72225.wav
e0ad96e1.wav
e0bfc4ec.wav
e0d34abd.wav
e0d8ea27.wav
e0d8ff62.wav
e0d96e55.wav
e0de96a6.wav
e0e373a5.wav
e0e6a1de.wav
e0eb0c8a.wav
e0ee247e.wav
e0fc67d9.wav
e103173b.wav
e10f3d3d.wav
e112e5a2.wav
e1145387.wav
e11bf16f.wav
e11ccb9d.wav
e1211dc4.wav
e122ef4b.wav
e128a888.wav
e13193af.wav
e1423cf1.wav
e14ebc15.wav
e15de084.wav
e15fedd2.wav
e1611806.wav
e173748a.wav
e19618a0.wav
e1ab90cd.wav
e1b1ee02.wav
e1c172fa.wav
e1c78f41.wav
e1ca89e3.wav
e1d88aa8.wav
e1eccf73.wav
e1f3b477.wav
e1fc13fd.wav
e218f5ab.wav
e21c7c53.wav
e22b1cc6.wav
e233206c.wav
e23393b9.wav
e245a3f5.wav
e2529ad3.wav
e25b3e5d.wav
e25f0efb.wav
e26962f7.wav
e269a6c9.wav
e269cdb6.wav
e26b56a0.wav
e275b9dd.wav
e27a0b20.wav
e291574a.wav
e29e1f8b.wav
e2adb002.wav
e2b92cb5.wav
e2b9b01a.wav
e2c0033d.wav
e2c7b0c7.wav
e2d0b667.wav
e2d30b3a.wav
e2d4c145.wav
e2de4941.wav
e2de57a7.wav
e2de89a3.wav

f2b6fd89.wav
f2ba99d8.wav
f2be783d.wav
f2bf7e84.wav
f2c60da6.wav
f2c781bc.wav
f2ce1323.wav
f2e678fe.wav
f2f5a41a.wav
f305a054.wav
f30f5fca.wav
f32309a8.wav
f324562c.wav
f330bf67.wav
f3368ed3.wav
f33acfe4.wav
f33fadfc.wav
f33fca42.wav
f342c01b.wav
f3482344.wav
f34b6f3d.wav
f34fb400.wav
f3512104.wav
f351c242.wav
f35775ce.wav
f35f18f6.wav
f363b68d.wav
f36cfcb5.wav
f36f1fc6.wav
f37780ed.wav
f3799674.wav
f37abe8d.wav
f37cc0fa.wav
f38c11c3.wav
f395c655.wav
f3967f91.wav
f3a871f0.wav
f3ab1ef3.wav
f3ac0e7d.wav
f3b36b54.wav
f3b615ee.wav
f3b7a589.wav
f3b85215.wav
f3bc7650.wav
f3c20960.wav
f3c8bebe.wav
f3ccf80f.wav
f3dc4e70.wav
f3e88606.wav
f3fb49b4.wav
f40ccfa4.wav
f40cdd0e.wav
f40d9111.wav
f40f792d.wav
f41ae5f1.wav
f4235f93.wav
f4272031.wav
f42b9f0e.wav
f4344d88.wav
f43c73b6.wav
f43f193a.wav
f442491d.wav
f4467979.wav
f446a833.wav
f44f2fc4.wav
f45176ff.wav
f471cb4d.wav
f485bb24.wav
f48a55d8.wav
f48be1f9.wav
f48ca77e.wav
f48e16cb.wav
f491595c.wav
f491aa38.wav
f492d14f.wav
f4964493.wav
f496c598.wav

In [38]:
np.save('../data_gen/test_mfcc_stretch=1.1_num=5_len=2.npy',test_data)
del test_data

In [39]:
X_train, y = prepare_data(train, config, '../input/audio_train/')

00044347.wav (0/9473)
001ca53d.wav (1/9473)
002d256b.wav (2/9473)
0033e230.wav (3/9473)
00353774.wav (4/9473)
003b91e8.wav (5/9473)
003da8e5.wav (6/9473)
0048fd00.wav (7/9473)
004ad66f.wav (8/9473)
0063ab88.wav (9/9473)
006f2f32.wav (10/9473)
0075d39c.wav (11/9473)
00780200.wav (12/9473)
0079d310.wav (13/9473)
0091fc7f.wav (14/9473)
0097160c.wav (15/9473)
00ad7068.wav (16/9473)
00c5808a.wav (17/9473)
00c82919.wav (18/9473)
00c934d7.wav (19/9473)
00c9e799.wav (20/9473)
00cb787c.wav (21/9473)
00ce569f.wav (22/9473)
00d1fe46.wav (23/9473)
00d3bba3.wav (24/9473)
00d40fa2.wav (25/9473)
00d9fa61.wav (26/9473)
00e2b4cd.wav (27/9473)
00f88dc5.wav (28/9473)
00fbb28b.wav (29/9473)
00fcbab2.wav (30/9473)
010aa387.wav (31/9473)
011a2185.wav (32/9473)
0120d246.wav (33/9473)
01235a12.wav (34/9473)
01257aad.wav (35/9473)
01302128.wav (36/9473)
013264d3.wav (37/9473)
013c3135.wav (38/9473)
01506d76.wav (39/9473)
015cf474.wav (40/9473)
0160d55e.wav (41/9473)
01638f61.wav (42/9473)
0172a2a5.wav (43/9473

09bc3033.wav (346/9473)
09ca1e09.wav (347/9473)
09ea5276.wav (348/9473)
0a037e96.wav (349/9473)
0a0a8d4c.wav (350/9473)
0a15b36b.wav (351/9473)
0a277f11.wav (352/9473)
0a2a5c05.wav (353/9473)
0a2b4c80.wav (354/9473)
0a32271b.wav (355/9473)
0a366772.wav (356/9473)
0a484e9f.wav (357/9473)
0a49afad.wav (358/9473)
0a54c770.wav (359/9473)
0a6bba04.wav (360/9473)
0a6dbf2c.wav (361/9473)
0a82b4d6.wav (362/9473)
0a8ac55d.wav (363/9473)
0a8d300c.wav (364/9473)
0a925754.wav (365/9473)
0a98104d.wav (366/9473)
0aa17642.wav (367/9473)
0aa32376.wav (368/9473)
0aa7edd0.wav (369/9473)
0aab722d.wav (370/9473)
0aad0a16.wav (371/9473)
0aaed52b.wav (372/9473)
0ac443ad.wav (373/9473)
0ac83c13.wav (374/9473)
0acba147.wav (375/9473)
0ad0bf22.wav (376/9473)
0ad5618e.wav (377/9473)
0ad666e3.wav (378/9473)
0ade0819.wav (379/9473)
0af32053.wav (380/9473)
0afca134.wav (381/9473)
0afcf36b.wav (382/9473)
0aff3cde.wav (383/9473)
0b05208c.wav (384/9473)
0b142b38.wav (385/9473)
0b1f22c3.wav (386/9473)
0b2a3ef0.wav (38

1397ddc8.wav (689/9473)
139c42cb.wav (690/9473)
13ac472f.wav (691/9473)
13b2325b.wav (692/9473)
13ca1116.wav (693/9473)
13da16ae.wav (694/9473)
13db0700.wav (695/9473)
13e15a49.wav (696/9473)
13ee6c44.wav (697/9473)
13f08982.wav (698/9473)
140027c6.wav (699/9473)
1401c7ff.wav (700/9473)
14041f9a.wav (701/9473)
141291fc.wav (702/9473)
1415b8b9.wav (703/9473)
14225f23.wav (704/9473)
142cb2c4.wav (705/9473)
142dac40.wav (706/9473)
14303ca5.wav (707/9473)
1430f062.wav (708/9473)
14344787.wav (709/9473)
143f37eb.wav (710/9473)
1441c25b.wav (711/9473)
14577fc0.wav (712/9473)
14683a1c.wav (713/9473)
14684ee8.wav (714/9473)
1468f23e.wav (715/9473)
146d5e25.wav (716/9473)
146fd7fd.wav (717/9473)
1472a525.wav (718/9473)
147f4395.wav (719/9473)
147f61ff.wav (720/9473)
14819552.wav (721/9473)
148a199b.wav (722/9473)
1498eb6e.wav (723/9473)
149ca673.wav (724/9473)
14a12f6c.wav (725/9473)
14ac7cbd.wav (726/9473)
14b489cd.wav (727/9473)
14b4f917.wav (728/9473)
14c22798.wav (729/9473)
14c476ad.wav (73

1c590c46.wav (1030/9473)
1c603b9f.wav (1031/9473)
1c67a5b9.wav (1032/9473)
1c6b07b8.wav (1033/9473)
1c720526.wav (1034/9473)
1c76a229.wav (1035/9473)
1c78595a.wav (1036/9473)
1c8943c0.wav (1037/9473)
1c8cddc2.wav (1038/9473)
1c9a423f.wav (1039/9473)
1ca0bd2f.wav (1040/9473)
1ca4d107.wav (1041/9473)
1cc13604.wav (1042/9473)
1cc55db4.wav (1043/9473)
1cccb873.wav (1044/9473)
1cd6919e.wav (1045/9473)
1ce38807.wav (1046/9473)
1ce4a00e.wav (1047/9473)
1ce70b78.wav (1048/9473)
1ced478c.wav (1049/9473)
1cf3ee46.wav (1050/9473)
1d08bd6d.wav (1051/9473)
1d0c860b.wav (1052/9473)
1d119cfc.wav (1053/9473)
1d1d0d72.wav (1054/9473)
1d23c74b.wav (1055/9473)
1d23fb6d.wav (1056/9473)
1d24518b.wav (1057/9473)
1d26c5e5.wav (1058/9473)
1d2808dd.wav (1059/9473)
1d28b9cb.wav (1060/9473)
1d395faa.wav (1061/9473)
1d3a05ab.wav (1062/9473)
1d491960.wav (1063/9473)
1d542616.wav (1064/9473)
1d5a6f5f.wav (1065/9473)
1d5d63d1.wav (1066/9473)
1d73907a.wav (1067/9473)
1d8502f5.wav (1068/9473)
1d85524c.wav (1069/9473)


256bc358.wav (1358/9473)
256ce8b0.wav (1359/9473)
25794ec7.wav (1360/9473)
258777c8.wav (1361/9473)
2588c966.wav (1362/9473)
258b6166.wav (1363/9473)
259c3671.wav (1364/9473)
25a3f8ba.wav (1365/9473)
25a4dda4.wav (1366/9473)
25a82ad1.wav (1367/9473)
25a8c380.wav (1368/9473)
25ad6ed7.wav (1369/9473)
25bca056.wav (1370/9473)
25c11b5d.wav (1371/9473)
25c89ec0.wav (1372/9473)
25c99ad7.wav (1373/9473)
25cc1f6a.wav (1374/9473)
25cd1d74.wav (1375/9473)
25eef09a.wav (1376/9473)
25f26095.wav (1377/9473)
25f4273e.wav (1378/9473)
260161b5.wav (1379/9473)
260da1b4.wav (1380/9473)
260ffd74.wav (1381/9473)
260fffaf.wav (1382/9473)
261ac300.wav (1383/9473)
261b8615.wav (1384/9473)
2620d32b.wav (1385/9473)
2624b75b.wav (1386/9473)
26262fd9.wav (1387/9473)
2627affd.wav (1388/9473)
2632db24.wav (1389/9473)
2634b73c.wav (1390/9473)
264301a3.wav (1391/9473)
264574a5.wav (1392/9473)
2647f749.wav (1393/9473)
2648b632.wav (1394/9473)
2655d941.wav (1395/9473)
2658f202.wav (1396/9473)
2659c14c.wav (1397/9473)


...

d6e1f1ac.wav (7923/9473)
d6f501d7.wav (7924/9473)
d6f56693.wav (7925/9473)
d6f85c72.wav (7926/9473)
d6ff6f09.wav (7927/9473)
d7050320.wav (7928/9473)
d7137371.wav (7929/9473)
d71cc17c.wav (7930/9473)
d7241122.wav (7931/9473)
d72830c2.wav (7932/9473)
d729c959.wav (7933/9473)
d72d685c.wav (7934/9473)
d7371221.wav (7935/9473)
d73be9a6.wav (7936/9473)
d73efc33.wav (7937/9473)
d75427d7.wav (7938/9473)
d760f427.wav (7939/9473)
d76bdc15.wav (7940/9473)
d77358d0.wav (7941/9473)
d773a9e6.wav (7942/9473)
d774a3f3.wav (7943/9473)
d77a30c4.wav (7944/9473)
d7857dfb.wav (7945/9473)
d78762a7.wav (7946/9473)
d788ea27.wav (7947/9473)
d791e73a.wav (7948/9473)
d79b9208.wav (7949/9473)
d7aae6fd.wav (7950/9473)
d7aef626.wav (7951/9473)
d7b31417.wav (7952/9473)
d7b32ed3.wav (7953/9473)
d7b534d3.wav (7954/9473)
d7baee61.wav (7955/9473)
d7d8b12a.wav (7956/9473)
d7d8c724.wav (7957/9473)
d7d9e3ad.wav (7958/9473)
d7dbcb1a.wav (7959/9473)
d7e50dfc.wav (7960/9473)
d7e56425.wav (7961/9473)
d7e5753e.wav (7962/9473)


df426834.wav (8252/9473)
df462a16.wav (8253/9473)
df61608d.wav (8254/9473)
df6223de.wav (8255/9473)
df630658.wav (8256/9473)
df66a03d.wav (8257/9473)
df6a940f.wav (8258/9473)
df73bad7.wav (8259/9473)
df7593da.wav (8260/9473)
df75d2e7.wav (8261/9473)
df79d259.wav (8262/9473)
df881a37.wav (8263/9473)
df890004.wav (8264/9473)
df8afeea.wav (8265/9473)
df91fbcc.wav (8266/9473)
df976153.wav (8267/9473)
df98be15.wav (8268/9473)
df9e6042.wav (8269/9473)
dfb39975.wav (8270/9473)
dfb42c74.wav (8271/9473)
dfb8afae.wav (8272/9473)
dfbeefb5.wav (8273/9473)
dfbf40dd.wav (8274/9473)
dfc4a855.wav (8275/9473)
dfce618e.wav (8276/9473)
dfd03330.wav (8277/9473)
dfd3ad5c.wav (8278/9473)
dfe13537.wav (8279/9473)
dfe7423a.wav (8280/9473)
dfed3be6.wav (8281/9473)
dffc9217.wav (8282/9473)
e0039925.wav (8283/9473)
e00d4ff7.wav (8284/9473)
e00ede63.wav (8285/9473)
e01a1ca0.wav (8286/9473)
e01f2d8b.wav (8287/9473)
e022b7b0.wav (8288/9473)
e030fb43.wav (8289/9473)
e03693c5.wav (8290/9473)
e0435db2.wav (8291/9473)


e81dd3c0.wav (8580/9473)
e81f1ed4.wav (8581/9473)
e820a056.wav (8582/9473)
e8285da5.wav (8583/9473)
e8358d8d.wav (8584/9473)
e8453e75.wav (8585/9473)
e84e08a8.wav (8586/9473)
e85715ab.wav (8587/9473)
e8582c7b.wav (8588/9473)
e85d162f.wav (8589/9473)
e8617ad4.wav (8590/9473)
e86bee0a.wav (8591/9473)
e86dbad2.wav (8592/9473)
e87e8024.wav (8593/9473)
e87f8984.wav (8594/9473)
e8874873.wav (8595/9473)
e887fb05.wav (8596/9473)
e888546b.wav (8597/9473)
e88d6949.wav (8598/9473)
e8924446.wav (8599/9473)
e8952741.wav (8600/9473)
e897db78.wav (8601/9473)
e898cffd.wav (8602/9473)
e89ebeb9.wav (8603/9473)
e8b24092.wav (8604/9473)
e8b706e7.wav (8605/9473)
e8cce8e8.wav (8606/9473)
e8d640b3.wav (8607/9473)
e8d6b248.wav (8608/9473)
e8dee9ee.wav (8609/9473)
e8e00924.wav (8610/9473)
e8e45406.wav (8611/9473)
e8ee43a1.wav (8612/9473)
e8f7bfb0.wav (8613/9473)
e8f862ba.wav (8614/9473)
e8fa13d1.wav (8615/9473)
e8fbaaf7.wav (8616/9473)
e8fd7f2f.wav (8617/9473)
e8fea86d.wav (8618/9473)
e9042c36.wav (8619/9473)


f093caf9.wav (8908/9473)
f09701aa.wav (8909/9473)
f09f18fe.wav (8910/9473)
f0a466a1.wav (8911/9473)
f0a743c5.wav (8912/9473)
f0b1afe7.wav (8913/9473)
f0b40e7a.wav (8914/9473)
f0bf7f04.wav (8915/9473)
f0bfdfae.wav (8916/9473)
f0c40412.wav (8917/9473)
f0c79f5e.wav (8918/9473)
f0d315bc.wav (8919/9473)
f0d644d1.wav (8920/9473)
f0dfe343.wav (8921/9473)
f0e34f4c.wav (8922/9473)
f0f17a5c.wav (8923/9473)
f0f48b44.wav (8924/9473)
f0f8d174.wav (8925/9473)
f1033230.wav (8926/9473)
f1062ecf.wav (8927/9473)
f10a5f1c.wav (8928/9473)
f10c7f27.wav (8929/9473)
f10ddf95.wav (8930/9473)
f11109de.wav (8931/9473)
f1119913.wav (8932/9473)
f114112a.wav (8933/9473)
f133df4c.wav (8934/9473)
f1352a21.wav (8935/9473)
f1403f83.wav (8936/9473)
f142df15.wav (8937/9473)
f1562014.wav (8938/9473)
f1573152.wav (8939/9473)
f1691e35.wav (8940/9473)
f1754aea.wav (8941/9473)
f1839e9f.wav (8942/9473)
f184970b.wav (8943/9473)
f18df301.wav (8944/9473)
f1923708.wav (8945/9473)
f19f12a6.wav (8946/9473)
f1a0a22a.wav (8947/9473)


f991a90c.wav (9237/9473)
f99b36b2.wav (9238/9473)
f9a4b662.wav (9239/9473)
f9a85ce0.wav (9240/9473)
f9adb074.wav (9241/9473)
f9afa20b.wav (9242/9473)
f9c0004a.wav (9243/9473)
f9c22f47.wav (9244/9473)
f9c912b9.wav (9245/9473)
f9c99cbd.wav (9246/9473)
f9d9ea85.wav (9247/9473)
f9ef92a3.wav (9248/9473)
f9f02b7d.wav (9249/9473)
f9f26b10.wav (9250/9473)
fa101519.wav (9251/9473)
fa2155fc.wav (9252/9473)
fa2786f2.wav (9253/9473)
fa32109e.wav (9254/9473)
fa3a3fea.wav (9255/9473)
fa3b1ef3.wav (9256/9473)
fa47fd75.wav (9257/9473)
fa481b8b.wav (9258/9473)
fa4d4a92.wav (9259/9473)
fa50f460.wav (9260/9473)
fa5e59d2.wav (9261/9473)
fa653b7a.wav (9262/9473)
fa65b546.wav (9263/9473)
fa67cfe3.wav (9264/9473)
fa68671d.wav (9265/9473)
fa6b48ee.wav (9266/9473)
fa71933a.wav (9267/9473)
fa722117.wav (9268/9473)
fa80fe22.wav (9269/9473)
fa842956.wav (9270/9473)
fa901492.wav (9271/9473)
fa9088a4.wav (9272/9473)
fa9f67e4.wav (9273/9473)
faa56a16.wav (9274/9473)
faa7e12e.wav (9275/9473)
fab57312.wav (9276/9473)


In [40]:
np.save('../data_gen/train_mfcc_stretch=1.1_num=5_len=2.npy',X_train)
np.save('../data_gen/label_stretch=1.1_num=5_len=2.npy',y)